In [1]:
!pip install torch torchvision -q

In [1]:
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

### Multi-class probplem

### Resnet training

In [6]:
!nvidia-smi

Wed May 24 15:40:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   23C    P8    14W / 370W |   3020MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
| 51%   

In [2]:
def clean():
    torch.cuda.empty_cache()
    gc.collect()

In [7]:
clean()

In [9]:
os.environ['CUDA_VISIABLE_DEVICES']='0'

In [3]:
device = torch.device('cuda')

### Efficientnet

In [135]:
from effnet.efficientnet import efficientnet

from utils.trainer import Trainer
from data.multi_class_build_data import build_dataloader

BATCH_SIZE=8

transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [136]:
criteriation=nn.CrossEntropyLoss()

In [144]:
effnet = efficientnet()
effnet.classifier[1] = nn.Linear(1280, NUM_CLASS)

In [145]:
effnet = effnet.to(device)

In [146]:
optimizer = torch.optim.Adam(effnet.parameters(), lr=1e-4)

### Training 

In [147]:
NAME = 'effnet'

trainer = Trainer(effnet, 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=2,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [148]:
trainer.training()

[1]/[2] Epoch starts
	 Batch train loss: 1.4173368215560913, accuracy 0.125
	 Batch train loss: 0.6500968933105469, accuracy 0.625
	 Batch train loss: 0.6747581362724304, accuracy 0.875
	 Batch train loss: 1.0785253047943115, accuracy 0.5
	 Batch train loss: 0.22702595591545105, accuracy 0.875
[1]/[2] End epoch: train loss: 0.693294952304126, val loss: 0.3635874504292515
	 Epoch train accuracy: 0.6995078921318054, val accuracy: 0.8468033775633294

[2]/[2] Epoch starts
	 Batch train loss: 0.24580885469913483, accuracy 0.875
	 Batch train loss: 0.37618398666381836, accuracy 0.875
	 Batch train loss: 0.242514967918396, accuracy 0.875
	 Batch train loss: 0.17320388555526733, accuracy 0.875
	 Batch train loss: 0.1986524909734726, accuracy 0.875
[2]/[2] End epoch: train loss: 0.2528687315517201, val loss: 0.3794699323693939
	 Epoch train accuracy: 0.908406138420105, val accuracy: 0.8584640128669079



### Evaluate model

In [149]:
effnet.load_state_dict(torch.load(f'multi_class_output/effnet.pt')['model_state_dict'])
effnet = effnet.to(device)

In [150]:
trainer = Trainer(effnet, 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [151]:
_, _, preds = trainer.val()

In [152]:
prediciton = pd.DataFrame([preds[0].cpu().numpy(), preds[1].cpu().numpy()]).T
prediciton.rename = ['labels', 'predictions']

prediciton.to_csv('multi_class_output/{name}.csv'.format(name=NAME), index=False)

### Multi-label problem

In [10]:
%cd /home/jovyan/work/

/home/jovyan/work


In [7]:
data = pd.read_csv('sample_labels.csv')
data['lables'] = data['Finding Labels'].str.split('|')

labels = []
for lable in data['lables'].values:
    labels.extend(lable)   
    
labels = pd.DataFrame(labels, columns=['labels'])

### Losses

In [ ]:
weights = 1 / (labels.value_counts()/ labels.shape[0])
weights = torch.tensor(weights.reset_index().sort_values(by='labels')[0].values)

class WeightedMultilabel(nn.Module):  
    def __init__(self, weights: torch.Tensor):  
        super(WeightedMultilabel, self).__init__()  
        self.cerition = nn.BCEWithLogitsLoss(reduction='none')  
        self.weights = weights  
  
    def forward(self, outputs, targets):  
        loss = self.cerition(outputs, targets)  
        return (loss * self.weights).mean()  

weights = weights.to(device)

criteriation = WeightedMultilabel(weights)

In [ ]:
from utils.trainer import Trainer
from data.multi_label_build_data import build_dataloader
from effnet.blocks import SamePadConv2d
from effnet.efficientnet import efficientnet


BATCH_SIZE=4

transfrom = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(30),
    transforms.ColorJitter(0.5),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [ ]:
del effnet
effnet = efficientnet()
effnet.classifier[1] = nn.Linear(1280, NUM_CLASS)

In [ ]:
effnet.features[0][0] = SamePadConv2d(1, 32, 3, 2)

In [ ]:
effnet = effnet.to(device)

In [ ]:
optimizer = torch.optim.Adam(effnet.parameters(), lr=1e-4)

In [ ]:
NAME = 'effnet'

trainer = Trainer(effnet , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=5,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [ ]:
trainer.training()

[1]/[5] Epoch starts
	 Batch train loss: 0.7092497944831848, accuracy 0.0761904761904762


KeyboardInterrupt: 

### Evaluate model

In [179]:
effnet.load_state_dict(torch.load(f'multi_label_output/effnet.pt')['model_state_dict'])
effnet = effnet.to(device)

In [180]:
trainer = Trainer(effnet , 
                  criteriation,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  path_output='multi_label_output/{name}.pt'.format(name=NAME),
                  multi_label=True
                 )

In [181]:
_, _, preds = trainer.val()

In [182]:
pd.DataFrame(preds[0].cpu().numpy()).to_csv('multi_label_output/true_{name}.csv'.format(name=NAME), index=False)
pd.DataFrame(preds[1].cpu().numpy()).to_csv('multi_label_output/pred_{name}.csv'.format(name=NAME), index=False)